In [1]:
import numpy as np
import math
from decimal import Context, setcontext, getcontext, Decimal, ROUND_HALF_EVEN

In [2]:
myothercontext = Context(prec=7)
setcontext(myothercontext)

In [3]:
#Setup 
Neutral = Decimal(1)
Annual_fix = Decimal(365/360)
Nper_in_a_year = Decimal(1/12) #Not valid in Peru and peruvians
Periods = Decimal(12) #Not valid in Peru and peruvians
Significant_numbers = 4

In [4]:
#to test
interest = Decimal(0.0125)  #anual
nper = 180
grace = Decimal(6)
insurance_monthly_fix = Decimal(3)
property_value = Decimal(5000)
loan = Decimal(4000)
unsolved_factor = Decimal(1)
credit_life_insurance_factor = Decimal(0.000068)
credit_fire_insurance_factor = Decimal(0.00168)

In [5]:
credit_property_insurance_factor = {
    "dpto": Decimal(0.9),
    "casa": Decimal(0.75),
}

In [6]:
#aux functions only
def thirdpow(a,b,c):
    return pow(pow(a,b),c) #Excel does a ^ b ^ c as [(a ^ b) ^ c] instead of [a ^ (b ^ c)]

def get_monthly_rate(annual_rate):
    getcontext().rounding = ROUND_HALF_EVEN
    return thirdpow(Neutral + annual_rate, Annual_fix, Nper_in_a_year ) - Neutral



def get_life_insurance(per, loan):
    first_period_value = 1
    default_period_value = 2
    return first_period_value if per == 1 else default_period_value



def get_monthly_rate_amount_applied(annual_rate, grace):
    return round(pow((Neutral + Decimal(annual_rate)), Decimal(unsolved_factor_k(grace)/Periods)) - Neutral, Significant_numbers)



class SimulatedPayment:
    def __init__(self, property_value, loan, nper, interest, grace, property_type):
        self.interest = interest
        self.grace = grace
        self.nper = nper
        self.loan = loan
        self.property_value = property_value
        self.property_type = property_type
        self.payment = self.get_payment()
        self.total = self.get_total()
                 
    def unsolved_factor_k(self, grace):
        unsolved_factor = Neutral
        for x in range(int(grace)):
            unsolved_factor += unsolved_factor * get_monthly_rate(interest)
        return unsolved_factor
    
    def get_payment(self):
        getcontext().rounding = ROUND_HALF_EVEN
        monthly_rate_aux = get_monthly_rate(self.interest)
        num = self.unsolved_factor_k(int(self.grace)) * monthly_rate_aux
        dem = Neutral - (pow((Neutral + monthly_rate_aux), self.nper * -Neutral))
        return num / dem * loan
    
    credit_property_insurance_factor = {
        "dpto": Decimal(0.9),
        "casa": Decimal(0.75),
    }
    
    def get_fire_insurance(self, per=1):
        fire_rate = credit_fire_insurance_factor * self.property_value * \
                    Nper_in_a_year * self.credit_property_insurance_factor[self.property_type]

        first_period_value = fire_rate * (grace + insurance_monthly_fix)
        return first_period_value if per == 1 else fire_rate
    
    def get_life_insurance(self, per=2):
        life_rate = self.unsolved_factor_k(self.grace) * self.loan * \
                             credit_life_insurance_factor 
        
        first_period_value = life_rate * (self.grace + insurance_monthly_fix)
        default_period_value = life_rate
        return first_period_value if per == 1 else default_period_value     
    
    def get_total(self):
        return self.get_payment() + self.get_fire_insurance()

    def __repr__(self):
        return f'SimulatedPayment{self.__dict__}'
    
credit = SimulatedPayment(property_value, loan, nper, interest, grace, "dpto")

print(credit.get_life_insurance())


0.2737180
